In [4]:
import googlemaps
from datetime import datetime

# load API key from file not included in repo
apikey = None
with open("./key.txt", 'r') as keyfile:
    apikey = keyfile.readline().strip()

tmode = "walking"

#print(apikey)
gmaps = googlemaps.Client(key=apikey)

# example call to directions
"""
now = datetime.now()
directions_result = gmaps.directions("Sydney Town Hall",
                                     "Parramatta, NSW",
                                     mode="transit",
                                     departure_time=now)
"""


#read in json to get address 
import json 


#this works eh
jsonData=None 
with open("bars.json","r") as iFile: 
    jsonData = json.loads(iFile.readline()) 
    
print(jsonData)



[{u'bar': u'ricks', u'address': u' 224 Abbot Rd, East Lansing, MI 48823'}, {u'bar': u'pts', u'address': u' 210 Abbott Rd., East Lansing, MI 48823'}, {u'bar': u'elAz', u'address': u'  225 Ann St, East Lansing, MI 48823'}, {u'bar': u'harpers', u'address': u' 131 Albert Ave, East Lansing, MI 48823'}, {u'bar': u'theRiv', u'address': u' 231 M. A. C. Ave, East Lansing, MI 48823'}, {u'bar': u'peanutBarrel', u'address': u'  521 E Grand River Ave, East Lansing, MI 48823'}, {u'bar': u'tinCan', u'address': u' 313 E Grand River Ave'}, {u'bar': u'crunchys', u'address': u' 254 E Grand River Ave, East Lansing, MI 48823'}, {u'bar': u'dublin', u'address': u' 327 Abbot Rd, East Lansing, MI 48823'}, {u'bar': u'louHas', u'address': u' 211 E Grand River Ave, MI 48823 '}, {u'bar': u'hopCat', u'address': u' 300 Grove St, East Lansing, MI 48823'}, {u'bar': u'fieldHouse', u'address': u' 213 Ann St, East Lansing, MI 48823'}, {u'bar': u'landshark', u'address': u' 101 E Grand River Ave, East Lansing, MI 48823'}, 